# 1. 투표 기반 학습기

- 앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘한다. 다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습하는 것입니다. 이렇게 하면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상시킵니다.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf),
                ('rf', rnd_clf),
                ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [3]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.872
SVC 0.888
VotingClassifier 0.896


/anaconda3/envs/base-workspace/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Voting 방식

- 다수결: voting='hard'
- 확률 평균: voting='soft'

# 2. Bagging과 페이스팅

- 다양한 분류기를 만드는 한 가지 방법은 각기 다른 알고리즘을 사용하는 것입니다. 또 다른 방법은 같은 알고리즘을 사용하지만 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습하는 것입니다.

- 훈련 세트에서 중복을 허용하여 샘플링하는 방식을 배깅이라 하며, 중복을 허용하지 않고 샘플링하는 방식을 pasting이라 한다.
- 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있다. 하지만 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있다. 이 샘플링과 훈련 과정은 다음과 같다.

## 1. 사이킷런의 배깅과 페이스팅

- 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만든다. 수집 함수는 전형적으로 분류일 떄는 통계적 최빈값이고 회귀에 대해서는 평균을 계산한다. 개별 예측기는 원본 훈련 세트로 훈련시키는 것보다 훨씬 크게 편향되어 있지만 수집 함수를 통과하면 편향과 분산이 모두 감소한다. 

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

- 배깅은 일반적으로 비슷한 편향에서 더 작은 분산을 만듭니다. (훈련 세트의 오차 수가 거의 비슷하지만 결정 경계는 덜 불규칙하다)
- 부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높습니다. 하지만 이는 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킵니다. 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호합니다.

### 2. oob 평가

- 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링 되고 어떤 것은 전혀 선택되지 않을 수 있습니다. BagginCLassifier는 기본갑승로 중복을 허용하여 훈련 세트의 크기 만큼 m개의 샘플을 선택합니다. 선택되지 않은 나머지 37%를 oob 샘플이라 합니다. 

- 예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 검증 세트나 교차 검증을 사용하지 않고 oob 샘플을 사용해 평가할 수 있습니다. 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻습니다.

- 사이킷런에서 BaggingClassifer를 만들 때 oob_score=True로 지정하면 훈련이 끝나면 자동으로 oob 평가를 수행합니다. 다음 코드는 이 과정을 보여줍니다. 평가 점수 결과는 oob_score_ 변수에 저장되어 있다.

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    bootstrap=True,
    n_jobs=-1,
    oob_score=True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.8986666666666666

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.912

In [7]:
bag_clf.oob_decision_function_

array([[0.41397849, 0.58602151],
       [0.32954545, 0.67045455],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.08510638, 0.91489362],
       [0.37931034, 0.62068966],
       [0.02463054, 0.97536946],
       [1.        , 0.        ],
       [0.98412698, 0.01587302],
       [0.75543478, 0.24456522],
       [0.        , 1.        ],
       [0.81018519, 0.18981481],
       [0.8556701 , 0.1443299 ],
       [0.95833333, 0.04166667],
       [0.06145251, 0.93854749],
       [0.00515464, 0.99484536],
       [0.97029703, 0.02970297],
       [0.97687861, 0.02312139],
       [0.99473684, 0.00526316],
       [0.01470588, 0.98529412],
       [0.34640523, 0.65359477],
       [0.9127907 , 0.0872093 ],
       [1.        , 0.        ],
       [0.97126437, 0.02873563],
       [0.        , 1.        ],
       [0.99459459, 0.00540541],
       [1.        , 0.        ],
       [0.00490196, 0.99509804],
       [0.66111111, 0.33888889],
       [0.

## 3. 랜덤 패치와 랜덤 서브스페이스

- BaggingClassifier는 특성 샘플링도 지원한다. max_featuers, bootstrap_features 두 매개변수로 조절된다. 작동 방식은 max_samples, bootstrap과 동일하지만 샘플이 아니고 특성에 대한 샘플링이다.
- 훈련 특성과 샘플을 모두 샘플링하는 것을 랜덤 패치 방식이라고 합니다.
- 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것을 랜덤 서브스페이스 방식이라 합니다.

특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춥니다.

# 4. 랜덤 포레스트

- 랜덤 프로세트는 일반적으로 배깅 방법을 적용한 결정 트리의 앙상블이다.

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_rped_rf = rnd_clf.predict(X_test)

- 랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위를 더 주입한다. 이는 결국 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어냅니다. 다음은 BaggingClassifier를 사용해 앞의 RandomForestClassifier와 거의 유사하게 만든 것이다.

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter='random',
                           max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

### 1. 엑스트라 트리

- 랜덤 포레스트에서는 트리를 만들 때 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용한다. 트
- 트리를 더욱 무작위로 만들기 위해 최적의 임계값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택한다. 이를 익시트림 랜덤 트리앙상블이라고 부른다.
- 여기에서도 편향이 늘지만 분산이 준다. 모든 노드마다 특성마다 가장 최적의 임계값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되므로 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠르다.

### 2. 특성 중요도

- 랜덤 포레스트는 어떤 특성을 사용한 노드가 평균적으로 불순도를 감소시키는 지 확인하여 특성의 중요돌르 측정한다.

In [10]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])

for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.10432961938769875
sepal width (cm) 0.0249237948056056
petal length (cm) 0.4566556087072346
petal width (cm) 0.4140909770994616


- 랜덤 포레스트는 특히 특성을 선택해야 할 때 어떤 특성이 중요한 지 빠르게 확인할 수 있어 매우 편리하다

# 5. 부스팅

- 부스팅은 약학 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말합니다. 부스팅 방법의 아이디어는 앞의 모델을 보완해나가면서 일련의 예측기를 학습해나간다. 부스팅 방법에는 여러 가지가 있지만 가장 인기 있는 것은 Adaboost, Gradient Boosting이다.

### 1. Adaboost

- 이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소 적합했던 훈련 샘플의 가중치를 더 높이는 것이다. 이렇게 하면 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 됩니다. 이것이 아다부스트에서 사용하는 방식이다.

- 경사 하강법은 비용 함수를 최소화 하기 위해 예측기의 모델 파라미터를 조정해가는 반면 아다부스틑 점차 더 좋아지도록 앙상블에 예측기를 추가합니다.

- 모든 예측기가 훈련을 마치면 이 앙상블은 배깅이나 페이스팅과 비슷한 방식으로 예측을 만듭니다. 하지만 가중치가 적용된 훈련 세트의 전반적인 정확도에 따라 예측기마다 다른 가중치가 적용됩니다.

### 알고리즘

1) 가중치 초기화: 샘플의 가중치를 모두 1/m으로 초기화한다.

2) 에러율 조정: j 번째 예측기의 가중치가 적용된 에러율
- rj = 틀린 샘플 가중치 합 / 전체 샘플 가중치 합

3) 예측기 가중치: aj = hyperparmeter * log(1-rj) / rj
- 올바르게 분류한 모델일 수록 가중치가 높다.

4) 샘플의 가중치 업데이트
- 잘못 분류된 샘플의 가중치가 증가한다.
- 정분류한 샘플의 경우 - w <- w
- 오분류한 샘플의 경우 w <- w * exp(aj)
- 이후 정규화

5) Adaboost 예측
- y = argmax(sigma(aj))

In [11]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

### 2. 그래디언트 부스팅

- 아다부스트는 반복마다 샘플의 가중치를 수정했지만 그래디언트 부스팅은 잔여 오차에 새로운 예츠기를 학습한다.

In [16]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1.0, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=3, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

- 아래 sklearn GBRT는 위에 앙상블과 같다.
- learning_rate가 각 트리의 기여정도를 조정한다. 이를 0.1 처럼 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아진다. 이를 축소(shrinkage)라고 부르는 규제 방법이다.
- 최적의 트리 수를 구하기 위해서는 조기 종료 기법을 사용할 수 있다. 간단하게 구현하려면 staged_predict() 메서드를 사용한다. 이 메서드는 훈련의 각 단계에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자를 반환한다. 다음의 코드는 120개의 트리로 GRBT 앙상블을 훈련시키고 최적 트리 수를 찾기 위해 각 훈련 단계에서 검증 오차를 측정한다. 마지막에 최적 트리 수를 사용해 새로운 GRBT 앙상블을 훈련한다.

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_esimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_esimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=53, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

- 많은 수의 트리를 먼저 훈련시키고 최적의 수를 찾기 위해 살펴보는 대신) 실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수 있다. warm_start=True로 설정하면 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해준다. 다음 코드는 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춘다.

In [19]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0

for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val) 
    val_error = mean_squared_error(y_val, y_pred)
    
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

Gradient Boosting Regressor는 각 트리가 훈련할 떄 사용한 훈련 샘플 비율을 지정할 수 있는 subsample 매개변수도 지원한다. 예를 들어 subsample=0.25라고 하면 각 트리는 무작위로 선택된 25% 훈련 샘플로 학습된다. 이런 기법을 **확률적 그래디언트 부스팅**이라 한다.

# 6. 스태킹

- 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수(직접 투표 같은)를 사용하는 대신 취합하는 모델을 훈련한다. 
- 아래의 세 예측기는 각각 다른 값을 예측하고 마지막 예측기(블렌더 또는 메타 학습기)가 이 예측을 입력으로 받아 최종 예측을 만든다.

- 블렌더를 학습시키는 일반적인 방법은 hold-out 세트를 사용하는 것이다. 먼저 훈련 세트를 두 개의 서브셋으로 나눈다. 
- 첫 번째 서브셋은 첫 번째 레이어의 예측을 훈련시키기 위해 사용된다. 그런 다음 첫 번째 레이어의 예측기를 사용해 두 번째 홀드아웃 세트에 대한 예측을 만든다. 예측기들이 훈련하는 동안 이 샘플을 전혀 못 봤기 떄문에 새로운 예측값이 만들어진다.
- 이제 홀드 아웃 세트의 각 샘플에 대해 세 개의 예측값이 만들어진다. 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들 수 있다. (새로운 훈련 세트는 3차원이 된다). 블렌더가 새 훈련 세트로 훈련됩니다. 즉 첫 번째 레이어의 예측을 지니고 타깃값을 예측하도록 학습된다.